In [2]:
import Pude_training_loop.model_training as model_training
import Pude_training_loop.dataset_loader as data_loader
import monocular_stereo_matching.stereo_pair_gen as stereo_pair_gen
import monocular_stereo_matching.model_training as stereo_model_training
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"


depth_anything_model, depth_anything_image_processor = model_training.get_model_image_processor_pair(model_name="depth_anything", model_path=model_training.models["depth_anything"], device=device)
dataset_loader = data_loader.DatasetLoader() # Initialize dataset loader with default parameters

#unimatch model

# Define training parameters
epochs = 3
learning_rate = 1e-5
batch_size = 1

# Define optimizer
optimizer = torch.optim.Adam(unimatch_model.parameters(), lr=learning_rate)

unimatch_model.train()

# seed the torch
torch.manual_seed(model_training.seed)

for epochs in range(epochs):
    for i in range(len(dataset_loader)):
        non_linear_image, linear_images = dataset_loader[i]

        non_linear_images_tensor = torch.tensor(non_linear_image, device=device)

        # Forward pass through the depth model
        depth_anything_output = model_training.get_model_output(model=depth_anything_model, 
                                                                image_processor=depth_anything_image_processor, 
                                                                raw_image=non_linear_images_tensor)
        
        # stereo pair generator
        image_2 = stereo_pair_gen(non_linear_image, depth_anything_output)

        # forward pass through the unimatch model
        unimatch_output = stereo_model_training(non_linear_images_tensor, image_2)

        # Compute loss
        loss = stereo_model_training.compute_loss(unimatch_output, linear_images)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        





OSError: Incorrect path_or_model_id: 'hr16/Unimatch/tree'. Please provide either the path to a local folder or the repo_id of a model on the Hub.